In [ ]:
import torch
print(torch.__version__)  # Kiểm tra phiên bản PyTorch
print(torch.cuda.is_available())  # Kiểm tra xem PyTorch có nhận GPU không
print(torch.version.cuda)  # Kiểm tra phiên bản CUDA mà PyTorch đang sử dụng
print(torch.cuda.get_device_name(0))  # Kiểm tra tên GPU


2.5.1+cu121
True
12.1
NVIDIA GeForce RTX 4060 Laptop GPU


In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from tqdm import tqdm

In [7]:
# Set seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)

# Define dataset paths
dataset_path = './Multi Cancer/Cervical Cancer'
cervix_categories = ['cervix_dyk', 'cervix_koc', 'cervix_mep', 'cervix_pab', 'cervix_sfi']

# Load dataset file paths and labels
filepaths, labels = [], []
for label, category in enumerate(cervix_categories):
    category_path = os.path.join(dataset_path, category)
    for file in os.listdir(category_path):
        filepaths.append(os.path.join(category_path, file))
        labels.append(label)

# Convert to DataFrame
df = pd.DataFrame({'filepath': filepaths, 'label': labels})

# Split dataset into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [8]:
# Custom dataset class
class CervicalCancerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['filepath']
        label = self.df.iloc[idx]['label']
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)  # Convert to PIL Image
        if self.transform:
            image = self.transform(image)
        return image, label

# Create data loaders
batch_size = 32
dataloaders = {
    'train': DataLoader(CervicalCancerDataset(train_df, transform), batch_size=batch_size, shuffle=True),
    'valid': DataLoader(CervicalCancerDataset(valid_df, transform), batch_size=batch_size, shuffle=False),
    'test': DataLoader(CervicalCancerDataset(test_df, transform), batch_size=batch_size, shuffle=False)
}

# Define model
class CervicalCancerModel(nn.Module):
    def __init__(self):
        super(CervicalCancerModel, self).__init__()
        self.base_model = models.resnet101(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, len(cervix_categories))
        )
    
    def forward(self, x):
        return self.base_model(x)

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CervicalCancerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)

c:\Users\tranh\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tranh\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Define model
class CervicalCancerModel(nn.Module):
    def __init__(self):
        super(CervicalCancerModel, self).__init__()
        self.base_model = models.resnet101(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, len(cervix_categories))
        )
    
    def forward(self, x):
        return self.base_model(x)

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CervicalCancerModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)

c:\Users\tranh\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tranh\anaconda3\envs\py310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:

# Training function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            correct = 0
            total = 0
            
            loop = tqdm(dataloaders[phase], desc=f'{phase.upper()} Epoch {epoch+1}/{num_epochs}', leave=True)
            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                correct += torch.sum(preds == labels.data)
                total += labels.size(0)
                
                loop.set_postfix(loss=running_loss/total, accuracy=correct.double()/total)
            
            epoch_loss = running_loss / total
            epoch_acc = correct.double() / total
            print(f'{phase.upper()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

train_model(model, dataloaders, criterion, optimizer)

# Save model
torch.save(model.state_dict(), "cervical_cancer_model.pth")


Epoch 1/10


TRAIN Epoch 1/10: 100%|██████████| 547/547 [02:25<00:00,  3.76it/s, accuracy=tensor(0.8681, device='cuda:0', dtype=torch.float64), loss=0.404]


TRAIN Loss: 0.4040 Acc: 0.8681


VALID Epoch 1/10: 100%|██████████| 118/118 [00:37<00:00,  3.13it/s, accuracy=tensor(0.9323, device='cuda:0', dtype=torch.float64), loss=0.201]


VALID Loss: 0.2007 Acc: 0.9323

Epoch 2/10


...

VALID Loss: 0.0230 Acc: 0.9952

Epoch 10/10


TRAIN Epoch 10/10: 100%|██████████| 547/547 [02:23<00:00,  3.80it/s, accuracy=tensor(0.9813, device='cuda:0', dtype=torch.float64), loss=0.056] 


TRAIN Loss: 0.0560 Acc: 0.9813


VALID Epoch 10/10: 100%|██████████| 118/118 [00:25<00:00,  4.56it/s, accuracy=tensor(0.9965, device='cuda:0', dtype=torch.float64), loss=0.0188]


VALID Loss: 0.0188 Acc: 0.9965
